In [1]:
import pybullet as p
import math
import time

pybullet build time: Oct 11 2021 21:00:24


In [8]:
physicsClientID = p.connect(p.GUI)

p.getConnectionInfo(physicsClientID)

error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.

In [9]:
import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())
# plane = p.loadURDF('plane.urdf')
StartPos = [0, 0, 3]
StartOrientation = p.getQuaternionFromEuler([1.57, 0, 0])
robot = p.loadURDF("humanoid.urdf", StartPos, StartOrientation, useFixedBase=1)
position, orientation = p.getBasePositionAndOrientation(robot)
numBodies = p.getNumBodies()

numJoints = p.getNumJoints(robot)
p.setGravity(0, 0, 0)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis chest

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis neck

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis right_shoulder

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis left_shoulder



In [4]:
for i in range(numJoints):
    if p.getJointInfo(robot, i)[2] == 0:
        print(p.getJointInfo(robot, i)[0], str(
            p.getJointInfo(robot, i)[1]).lstrip('b'))

4 'right_elbow'
7 'left_elbow'
10 'right_knee'
11 'right_ankle'
12 'left_hip'
13 'left_knee'
14 'left_ankle'


In [5]:
def joint_control_pos(joint_index, target_angle, bodyID=robot):

    p.setRealTimeSimulation(1)
    p.setJointMotorControl2(bodyIndex=bodyID,
                            jointIndex=joint_index,
                            controlMode=p.POSITION_CONTROL,
                            targetPosition=target_angle)

    # p.setRealTimeSimulation(0)


In [10]:
def VMAX(jointIndex, timePeriod, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[8]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[9]
    Vmax = -1*((upperLimit - lowerLimit) * math.pi) / timePeriod
    return Vmax

def velocity(timePeriod,Vmax,t):
    frequency = 2*math.pi/timePeriod
    return Vmax*math.sin(frequency*(t))

def joint_control_vel_sin(joint_index, timePeriod = 1, bodyID=robot):
    t=0
    timeStep = 0.01
    maxForce = 1
    vmax = VMAX(joint_index,timePeriod, bodyID)
    p.setRealTimeSimulation(1)
    while(t<=timePeriod):
        targetVel = velocity(timePeriod,vmax,t)
        if(targetVel==0):
            maxForce = 0
        else:
            maxForce = 1*targetVel/abs(targetVel)
        p.setJointMotorControl2(bodyID, joint_index, p.VELOCITY_CONTROL, targetVel, maxForce)
        time.sleep(timeStep)
        t += timeStep


In [25]:
p.setRealTimeSimulation(0)

In [46]:
p.setJointMotorControl2(robot, 9, p.POSITION_CONTROL, 0, 100)

In [11]:
joint_control_vel_sin(10)
p.getJointState(robot,10)

(-0.004164704970622795, 1.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.507558814033154)

In [6]:
joint_control_pos(10, -1.57)


In [32]:
joint_control_pos(10,0)
#reset joint angle

In [7]:
p.resetSimulation()
# go to urdf loading cell